In [15]:
np.arange(4)

array([0, 1, 2, 3])

In [16]:
import numpy as np
import pandas as pd

def as_colvec(x):   
    if (x.ndim == 2):
        return x
    else:
        return np.expand_dims(x,axis=1)

In [17]:
as_colvec (np.arange(4))

array([[0],
       [1],
       [2],
       [3]])

In [18]:
def implied_returns(delta,sigma,w):    #delta= risk aversion coefficient, sigma= var covar matrix, w=portfolio weights
    ir = delta* sigma.dot(w).squeeze()
    ir.name = 'Implied Returns'
    return ir

In [19]:
def proportional_prior(sigma,tau,p):
    helit_omega = p.dot(tau*sigma).dot(p.T)
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)

In [20]:
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q, 
       omega = None,
       delta=7.5, tau=.20):

    if omega is None:
         omega = proportional_prior(sigma_prior, tau, p)
    
    N = w_prior.shape[0]
    
    K = q.shape[0]
    
    pi = implied_returns(delta, sigma_prior, w_prior)
    
    sigma_prior_scaled = tau * sigma_prior
    
    nu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T)+ omega).dot(q-p.dot(pi).values))
    ##nu_bl = pi + (p.T).dot(sigma_prior_scaled).dot(inv(sigma_prior_scaled.dot(p)).dot(p.T)+ omega).dot(q-p.dot(pi).values)
    #nu_bl = pi + (p.T)@(sigma_prior_scaled)@(inv(sigma_prior_scaled@(p))@(p.T)+ omega)@(q-p@(pi).values)

    
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T)+ omega)).dot(p).dot(sigma_prior_scaled)
    ##sigma_bl = sigma_prior + sigma_prior_scaled - (p.T).dot(sigma_prior_scaled).dot(inv(sigma_prior_scaled.dot(p).dot(p.T)+ omega)).dot(sigma_prior_scaled).dot(p)
    
    #sigma_bl = sigma_prior + sigma_prior_scaled - (p.T)@(sigma_prior_scaled)@(inv(sigma_prior_scaled@ p @(p.T)+ omega))@(sigma_prior_scaled)@p

    return (nu_bl, sigma_bl)


In [21]:
prices = pd.read_excel('prices.xlsx',header=0,index_col=0, parse_dates= True, usecols="A:H")
returns = prices.pct_change()
returns = returns.dropna()

In [22]:
#returns= pd.DataFrame(returns)

#tickers = returns[:0] 
tickers = ['Euro Gov', 'Greek Gov', 'EU Corporate', 'EU HY' , 'Eur Eq', 'US Eq', 'Cash']
s = returns.cov()*52
s = np.matrix(s)
s = pd.DataFrame(s,index=tickers, columns=tickers)

In [23]:

#input the weights of the benchmark
WEuroGov = .30
WGreekGov = .10
WEUCorporate = .20
WEUHY = .05
WEurEq = .10
WUSEq = .10
WCash = .15

weights = pd.Series([WEuroGov, WGreekGov, WEUCorporate, WEUHY, WEurEq, WUSEq, WCash],index= tickers)
#weights = np.matrix(weights)
#weights = pd.DataFrame(weights)

pi=implied_returns(delta=7.5,sigma=s, w=weights)
pi


Euro Gov        0.004326
Greek Gov       0.069389
EU Corporate    0.002424
EU HY           0.017458
Eur Eq          0.052837
US Eq           0.045285
Cash           -0.000062
Name: Implied Returns, dtype: float64

In [24]:
def inverse(d):
    
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    
    w = inverse(sigma).dot(mu)
    
    if scale:
        w= w/sum(w)
        return w
    

In [25]:
#input the expectations
EEuroGov = .50
EGreekGov = .10
EEUCorporate = .20
EEUHY = .05
EEurEq = .05
EUSEq = .05
ECash = .0

mu_exp = pd.Series([EEuroGov, EGreekGov, EEUCorporate, EEUHY, EEurEq, EUSEq, ECash], index=tickers)
np.round(w_msr(s, mu_exp)*100,2)

Euro Gov        33.83
Greek Gov       -1.33
EU Corporate    74.64
EU HY          -10.24
Eur Eq          -0.03
US Eq            3.97
Cash            -0.83
dtype: float64

In [26]:
q= pd.Series(mu_exp)

p= pd.DataFrame([
    {'Euro Gov':1, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':1, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':1, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':1, 'Eur Eq':0, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':1, 'US Eq':0, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':1, 'Cash':0},
    {'Euro Gov':0, 'Greek Gov':0, 'EU Corporate':0, 'EU HY':0, 'Eur Eq':0, 'US Eq':0, 'Cash':1}
    ])


In [27]:
bl_mu, bl_sigma = bl(w_prior= weights, sigma_prior=s, p=p, q=q)

bl_mu

Euro Gov        0.292083
Greek Gov       0.282988
EU Corporate    0.121205
EU HY           0.122930
Eur Eq          0.039985
US Eq          -0.049595
Cash           -0.000171
dtype: float64

In [28]:
w_msr(bl_sigma, bl_mu)

Euro Gov        0.331657
Greek Gov      -0.004294
EU Corporate    0.586814
EU HY          -0.023729
Eur Eq          0.002434
US Eq           0.009402
Cash            0.097717
dtype: float64